In [15]:
# imports
import pandas as pd
import sqlite3
import pathlib
from sqlalchemy import create_engine, select, Column, Integer, String, Boolean
from sqlalchemy.orm import declarative_base, sessionmaker 

# make database directory, ignored by .gitignore
DB_PATH = pathlib.Path("databases")
DB_PATH.mkdir(exist_ok=True)
db_path = DB_PATH / "database.db"

# make uploads directory, ignored by .gitignore
UPLOADS_PATH = pathlib.Path("uploads")
UPLOADS_PATH.mkdir(exist_ok=True)
file_path = UPLOADS_PATH / "sample-data.csv"

# NOTE: NEED TO UPLOAD SAMPLE CSV DATA

In [3]:
# preprocessing
df = pd.read_csv(file_path)
print(f"Columns: {df.columns.tolist()}")

def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Normalize column names
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(' ', '_')
          .str.replace(r'[^0-9a-z_]', '', regex=True)
    )

    # Drop the "id" column if it exists to prevent issues with the ORM
    df = df.drop(columns=["id"], errors="ignore")
    return df
df_cleaned = preprocess_df(df)

Columns: ['Binder Name', 'Binder Type', 'Name', 'Set code', 'Set name', 'Collector number', 'Foil', 'Rarity', 'Quantity', 'ManaBox ID', 'Scryfall ID', 'Purchase price', 'Misprint', 'Altered', 'Condition', 'Language', 'Purchase price currency']


In [4]:
# sqlalchemy
engine = create_engine(f"sqlite:///{str(db_path.resolve())}")

Base = declarative_base()

class Collection(Base):
    __tablename__ = "collection"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    set_code = Column(String)
    collector_number = Column(String)
    foil = Column(String)
    rarity = Column(String)
    quantity = Column(Integer)
    scryfall_id = Column(String)
    language = Column(String)

def replace_collection_from_df(df: pd.DataFrame, engine):
    """
    Ensure ORM schema is authoritative: drop then create the table.
    So pandas doesn't alter the schema inadvertently.
    """
    Base.metadata.drop_all(engine, tables=[Collection.__table__])
    Base.metadata.create_all(engine)

    # Align DataFrame columns with ORM
    orm_cols = [c.name for c in Collection.__table__.columns if c.name != "id"]
    df = df.reindex(columns=orm_cols)
    
    df.to_sql("collection", con=engine, if_exists="append", index=False)

replace_collection_from_df(df_cleaned, engine)
Session = sessionmaker(bind=engine)

In [16]:
with Session() as session:
    stmt = select(Collection).filter(Collection.name.like('%Orcish%')).limit(10)
    results = session.scalars(stmt).all()
[r for r in results]